In [1]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn import model_selection

In [2]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

### Адаптивный бустинг на деревьях решений

In [3]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [4]:
def adaboost(X, y, N):

    n_objects = len(X)

    n_classes = len(np.unique((y)))

    w = np.ones(n_objects) / n_objects

    models = []

    for n in range(N):
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        if e >= 1 - 1/n_classes: 
            break

        alpha = 0.5 * np.log((1 - e) / e)

        match = predictions == y

        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        w /= w.sum()

        models.append((alpha, clf))
    
    return models

In [5]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        y_pred[range(n_objects), prediction] += alpha
    
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [6]:
N = 50

models = adaboost(X_train, y_train, N)

In [7]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 94.836


In [8]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 93.007


### Адаптивный бустинг на деревьях решений с применением метода главных компонент 

In [9]:
# Для начала отмасштабируем выборку
X_ = X_train.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [10]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(30,1), eig_pairs[1][1].reshape(30,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.21871855  0.23722283]
 [ 0.11672104  0.04890103]
 [ 0.22736145  0.21954968]
 [ 0.21974644  0.23525889]
 [ 0.13431478 -0.19137792]
 [ 0.23727232 -0.15632374]
 [ 0.25843206 -0.06874562]
 [ 0.26338589  0.03377092]
 [ 0.13734595 -0.19448438]
 [ 0.0621606  -0.36085979]
 [ 0.20655075  0.10178611]
 [ 0.01895747 -0.10705188]
 [ 0.20916554  0.0875406 ]
 [ 0.20416062  0.15342269]
 [ 0.00714823 -0.19564304]
 [ 0.16717355 -0.23118605]
 [ 0.1486753  -0.19944131]
 [ 0.1859253  -0.12920531]
 [ 0.03046398 -0.17290302]
 [ 0.10724175 -0.27178134]
 [ 0.22892005  0.21951474]
 [ 0.11540267  0.02752108]
 [ 0.23709879  0.20120702]
 [ 0.22565765  0.21851911]
 [ 0.11716547 -0.17756344]
 [ 0.20939731 -0.14975584]
 [ 0.22865312 -0.10955656]
 [ 0.25443096  0.00584962]
 [ 0.11811768 -0.1386789 ]
 [ 0.13554909 -0.27464872]]


In [11]:
new_X_train=X_.dot(W)

In [12]:
new_X_test=X_test[:, :]

In [13]:
# центрирование - вычитание из каждого значения среднего по строке
for j in range(cols):
    new_X_test[:, j] -= means[j]

# деление каждого значения на стандартное отклонение
for i in range(cols):
    new_X_test[:, i] /= std[i]

In [14]:
new_X_test=new_X_test.dot(W)

In [15]:
N = 50 

models = adaboost(new_X_train, y_train, N)

In [16]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(new_X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 93.662


In [17]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(new_X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 93.007


### Сравнение результатов

На этом датасете для адаптивного бустинга на деревьях решений получаем примерно такие же результаты при применении метода главных компоннент.